In [19]:
print('Hello LangChain')

Hello LangChain


In [20]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 


load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [21]:
# prompt
prompt = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 개발자입니다.") , 
     ("user", "{input}") ]
)
print(prompt)

prompt_text = prompt.format(input="자바는 무엇인가요? 자세하게 설명해주세요")

input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 개발자입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


In [22]:
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x000001784F471C10> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001784F472870> root_client=<openai.OpenAI object at 0x000001784F4726F0> root_async_client=<openai.AsyncOpenAI object at 0x000001784F40A360> model_name='meta-llama/llama-4-scout-17b-16e-instruct' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


### LCEL
* Prompt + LLM을 Chain으로 연결하기

In [23]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("" \
"You are an expert in AI Expert." \
"Answer the question. <Question>: {input}에 대해 쉽게 설명해주세요.")
prompt

PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are an expert in AI Expert.Answer the question. <Question>: {input}에 대해 쉽게 설명해주세요.')

In [24]:
# chain 연결 (LCEL)
chain = prompt | llm
print(type(chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [ ]:
from langchain_core.output_parsers import StrOutputParser

# chain 연결 (LCEL) prompt + llm + outputparser
output_parser = StrOutputParser()

chain2 = prompt | llm | output_parser
print(type(chain2))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [26]:
# chain 호출
try:
    result = chain.invoke({"input": "인공지능 모델의 학습 원리"})
    print(type(result))
    print(result.content)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'langchain_core.messages.ai.AIMessage'>
인공지능 모델의 학습 원리는 사람의 뇌가 학습하는 방식과 유사합니다. 컴퓨터가 데이터를 통해 배우고, 패턴을 발견하며, 미래의 새로운 데이터에 대해 예측하거나 결정을 내릴 수 있도록 하는 것입니다.

쉽게 설명하면 다음과 같습니다:

1. **데이터 수집**: 인공지능 모델을 학습시키기 위해 많은 데이터를 수집합니다. 이 데이터는 과거의 경험이나 관찰을 통해 얻어진 정보입니다.

2. **데이터 처리**: 수집된 데이터를 처리하고 준비하여 모델이 학습할 수 있는 형태로 만듭니다.

3. **모델 훈련**: 준비된 데이터를 바탕으로 인공지능 모델을 훈련시킵니다. 이 과정에서는 모델이 데이터의 패턴이나 관계를 발견하고 학습하도록 합니다.

4. **예측 또는 결정**: 훈련된 모델을 사용하여 새로운 데이터에 대해 예측하거나 결정을 내립니다.

5. **성능 평가**: 모델의 성능을 평가하고, 필요하다면 모델을 수정하거나 추가적인 학습을 진행합니다.

이를 비유하면, 어린 아이가 사물을 인식하고 이름을 배우는 과정과 유사합니다. 처음에는 많은 사물을 보여주고(데이터 수집), 그것이 무엇인지 설명해 줍니다(데이터 라벨링). 아이는 이것들을 반복적으로 보고 들으면서 패턴을 배우고, 나중에 새로운 사물을 만나면 이전에 배운 패턴을 바탕으로 그것이 무엇인지 인식하게 됩니다(예측 또는 결정).

인공지능 모델도 이와 비슷하게 데이터를 통해 학습하고, 그 학습을 바탕으로 새로운 상황에 대처하거나 예측을 하게 됩니다.


In [29]:
# chain 호출
try:
    result = chain2.invoke({"input": "LangChain의 Products(제품)는 어떤 것들이 있나요? 예를 들어 LangSmith, LangServe같은 products가 있어."})
    print(type(result))
    print(result)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'str'>
LangChain은 인공지능(AI) 모델을 활용하여 자연어 처리(NLP) 및 언어 생성 tasks를 지원하는 플랫폼입니다. LangChain의 주요 제품은 다음과 같습니다:

1. **LangSmith**: LangSmith는 LangChain에서 제공하는 통합 개발 환경(IDE)입니다. LangSmith를 사용하면 AI 모델을 쉽게 개발, 테스트 및 배포할 수 있습니다. LangSmith는 LangChain의 다양한 기능을 통합하여 사용자가 AI 모델을 쉽고 빠르게 구축할 수 있도록 지원합니다.

2. **LangServe**: LangServe는 LangChain에서 제공하는 API 서비스입니다. LangServe를 사용하면 사전에 구축된 AI 모델을 쉽게 API로 배포하고 사용할 수 있습니다. LangServe는 사용자가 LangChain의 다양한 AI 모델을 활용하여 자신만의 애플리케이션을 개발할 수 있도록 지원합니다.

LangChain의 다른 주요 제품 및 기능은 다음과 같습니다:

* **LangChain Studio**: LangChain Studio는 LangChain에서 제공하는 모델 개발 툴입니다. LangChain Studio를 사용하면 사용자가 자신의 AI 모델을 개발하고 테스트할 수 있습니다.

* **LangChain Hub**: LangChain Hub는 LangChain에서 제공하는 모델 공유 플랫폼입니다. LangChain Hub를 사용하면 다른 사용자가 개발한 AI 모델을 쉽게 찾아서 사용할 수 있습니다.

* **LangChain Agents**: LangChain Agents는 LangChain에서 제공하는 자율 에이전트입니다. LangChain Agents를 사용하면 AI 모델을 활용하여 다양한 작업을 자율적으로 수행할 수 있습니다.

이러한 제품과 기능을 통해 LangChain은 사용자들이 AI 모델을 쉽고 빠르게 개발, 배포 및 활용할 수 있도록 지원합니다.
